In [ ]:
import math as m
import random

import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Rectangle

import ipywidgets as widgets

In [ ]:
EXT_SIZE_X, EXT_SIZE_Y = 80, 60

INT_OFFS_X, INT_OFFS_Y = 20, 20
INT_SIZE_X, INT_SIZE_Y = 20, 20

EXT_BORDER = +300.0
INT_BORDER = +320.0

STEP = 1

def func(x, z):
    # return 30
    f0, beta = 60, 0.1
    x0, z0 = 45, 9
    return f0 * m.exp(-beta * ((x - x0)**2 + (z - z0)**2))

K = 20

In [ ]:
STEPS_1X = int(INT_OFFS_X / STEP)
STEPS_2X = int((INT_SIZE_X + 1) / STEP)
STEPS_3X = EXT_SIZE_X - INT_OFFS_X - INT_SIZE_X

STEPS_1Y = int(INT_OFFS_Y / STEP)
STEPS_2Y = int((INT_SIZE_Y + 1) / STEP)
STEPS_3Y = EXT_SIZE_Y - INT_OFFS_Y - INT_SIZE_Y

STEPS_X = STEPS_1X + STEPS_2X + STEPS_3X
STEPS_Y = STEPS_1Y + STEPS_2Y + STEPS_3Y

In [ ]:
# Matrix and accessors

w_matrix = np.zeros((STEPS_Y, STEPS_X), dtype=float)
h, w = w_matrix.shape

ibx0, iby0 = INT_OFFS_X, INT_OFFS_Y
ibx1, iby1 = INT_OFFS_X + INT_SIZE_X, INT_OFFS_Y + INT_SIZE_Y


def is_ext_bound(y, x):
    return x in [0, w - 1] or y in [0, h - 1]

def _is_hole_or_bound(y, x):
    return x >= ibx0 and x <= ibx1 and y >= iby0 and y <= iby1

def _is_int_bound_1dim(y, x):
    return x in [ibx0, ibx1] or y in [iby0, iby1]

def is_int_bound(y, x):
    return _is_hole_or_bound(y, x) and _is_int_bound_1dim(y, x)

def is_bound(y, x):
    return is_int_bound(y, x) or is_ext_bound(y, x)

def is_hole(y, x):
    return _is_hole_or_bound(y, x) and not _is_int_bound_1dim(y, x)

def is_target(y, x):
    return not (_is_hole_or_bound(y, x) or is_ext_bound(y, x))


def get_range1x():
    return range(1, ibx0)

def get_range2x():
    return range(ibx0, ibx1 + 1)

def get_range3x():
    return range(ibx1 + 1, w - 1)

def get_range1y():
    return range(1, iby0)

def get_range2y():
    return range(iby0, iby1 + 1)

def get_range3y():
    return range(iby1 + 1, h - 1)

In [ ]:
# Borders setup

for j in range(h):
    w_matrix[j, 0] = EXT_BORDER
    w_matrix[j, w - 1] = EXT_BORDER
for i in range(w):
    w_matrix[0, i] = EXT_BORDER
    w_matrix[h - 1, i] = EXT_BORDER

for j in get_range2y():
    w_matrix[j, ibx0] = INT_BORDER
    w_matrix[j, ibx1] = INT_BORDER
for i in get_range2x():
    w_matrix[iby0, i] = INT_BORDER
    w_matrix[iby1, i] = INT_BORDER

In [ ]:
# Function values setup

def FUNC2(x, z): return func(x, z) / K

for j in range(1, h - 1):
    for i in range(1, w - 1):
        if not is_target(i, j):
            continue
        w_matrix[j, i] = FUNC2(j * STEP, i * STEP)


In [ ]:
# Monte-carlo method (fixed)

ITERS_COUNT = 200

def prob_trace(j, i):
    res = 0
    pts_cnt = 0
    while not is_bound(j, i):
        res += w_matrix[j, i] * 0.25
        pts_cnt += 1

        dir = np.random.randint(0, 4)
        if dir % 2 == 0:
            j += 1 if dir < 2 else -1
        else:
            i += 1 if dir < 2 else -1
    res += w_matrix[j, i]
    return res

def prob_point(j, i):
    res = 0
    for _ in range(ITERS_COUNT):
        res += prob_trace(j, i)
    res /= ITERS_COUNT
    return res

In [ ]:
TARGET = (45, 9)
res = prob_point(*TARGET)
print('Res: {:.3f}'.format(res))
orig = 330.6864168849065
print('Err: {:.3f}%'.format((abs(res - orig) / orig) * 100))